## 说明

* 处理完数据就保存起来，然后把处理数据的代码注释掉
* os相关函数：
    * os.path
    * os.listdir(dirname)
    * os.walk
    * os.scandir

*  https://blog.csdn.net/happyjxt/article/details/51063854
* https://blog.csdn.net/qq_39839807/article/details/104070761

## 保存路径
```
path = '10captcha/images/'
num = 100 # 验证码图片数量

if not os.path.exists(path):
    os.makedirs(path)
```

## 1 目录中的子文件操作`os.walk('data/images/')`
* 获取文件名（经常文件名就是对应标签）

In [12]:
import os
from PIL import Image
import matplotlib.pyplot as plt

for root,dirs,files in os.walk('data/images/'): # 文件夹路径、文件夹名称和文件名
    print(root,dirs,files)
    for file in files:
        print(os.path.join(root,file))
        # image_path = os.path.join(root, file)
        # print(image_path)
        # img = Image.open(image_path)
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()

data/images/ [] ['cat.jpg', 'dao2.jpg', 'Girl.jpg']
data/images/cat.jpg
data/images/dao2.jpg
data/images/Girl.jpg


## 获取url特定数据

In [17]:
# inception模型下载
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
# 模型存放地址
inception_pretrain_model_dir = "inception_model"
# 获取文件名以及文件路径
filename = inception_pretrain_model_url.split('/')[-1]
print(filename)
filepath = os.path.join(inception_pretrain_model_dir,filename)
print(filepath)

inception-2015-12-05.tgz
inception_model\inception-2015-12-05.tgz


## 获取文件名为一个数组 `os.listdir(dataset_dir)`


In [41]:
import os
# 获取所以验证码图片
def _get_filenames(dataset_dir):
    captcha_names = []
    for filename in os.listdir(dataset_dir):
        # 获取文件路径
        print(filename.split('.')[-2])
        path = os.path.join(dataset_dir, filename)
        print(path)
        captcha_names.append(path)
    return captcha_names

dataset_dir = 'data/code'
_get_filenames(dataset_dir)

0131
data/code\0131.jpg
0153
data/code\0153.jpg
0161
data/code\0161.jpg
0178
data/code\0178.jpg
0398
data/code\0398.jpg


['data/code\\0131.jpg',
 'data/code\\0153.jpg',
 'data/code\\0161.jpg',
 'data/code\\0178.jpg',
 'data/code\\0398.jpg']

## 已得文件数组，获取文件label 
* 获取文件名：`filename.split('/')[-1][:4]`


In [37]:
def _convert_dataset(split_name, filenames):
    labelsAll = []
    for i,filename in enumerate(filenames):
        try:
            # 拿到4位验证码并encode()
            print(i)
            labels = filename.split('/')[-1][:4]
            labelsAll.append(labels)
            # print(labelsAll)
            # 生成protocal数据类型
            print(int(labels[0]), int(labels[1]), int(labels[2]), int(labels[3]))
        except IOError as e:
            print('Wrong: ' + filename)
            print('Error: ',e)
            print('Skip it\n')
    return labelsAll


# 运行函数
filenames = ['data/code/0131.jpg',
 'data/code/0153.jpg',
 'data/code/0161.jpg',
 'data/code/0178.jpg',
 'data/code/0398.jpg']
labelsAll = _convert_dataset('test', filenames)
print(labelsAll)

0
0 1 3 1
1
0 1 5 3
2
0 1 6 1
3
0 1 7 8
4
0 3 9 8
['0131', '0153', '0161', '0178', '0398']


In [ ]:
# 获取文件名（经常文件名就是对应标签）

# 获取文件mfcc特征和对应标签
def extract_features(wav_files):
    inputs = []
    labels = []
    
    for wav_file in tqdm(wav_files): # 循环每个文件
        # 读入音频文件
        audio,fs = librosa.load(wav_file) # audio--文件,fs--采样率 

        # 获取音频mfcc特征
        # [n_steps, n_inputs]  n_steps--几秒音频中分的帧数, n_inputs--特征
        mfccs = np.transpose(librosa.feature.mfcc(y=audio, sr=fs, n_mfcc=FLAGS.n_inputs), [1,0]) 
        inputs.append(mfccs.tolist())
    #获取label
    for wav_file in wav_files:
        label = wav_file.split('/')[-1].split('-')[1] # -分隔得到第2个数字
        labels.append(label) 
    return inputs, np.array(labels, dtype=np.int)


## 2 单文件中内容操作(每行  定位内容)

In [3]:
# 获得训练用的wav文件路径列表  
def get_wav_files(parent_dir,sub_dirs): 
    wav_files = []  
    for l, sub_dir in enumerate(sub_dirs):
        wav_path = os.path.join(parent_dir, sub_dir)
        for (dirpath, dirnames, filenames) in os.walk(wav_path):  
            for filename in filenames:  
                if filename.endswith('.wav') or filename.endswith('.WAV'):   # 判断后缀名是否为wav WAV
                    filename_path = os.sep.join([dirpath, filename])  
                    wav_files.append(filename_path)  
    return wav_files  

# 获取文件mfcc特征和对应标签
def extract_features(wav_files):
    inputs = []
    labels = []
    
    for wav_file in tqdm(wav_files):
        # 读入音频文件
        audio,fs = librosa.load(wav_file)

        # 获取音频mfcc特征
        # [n_steps, n_inputs]
        mfccs = np.transpose(librosa.feature.mfcc(y=audio, sr=fs, n_mfcc=FLAGS.n_inputs), [1,0]) 
        inputs.append(mfccs.tolist()) 
    #获取label
    for wav_file in wav_files:
        label = wav_file.split('/')[-1].split('-')[1]
        labels.append(label) 
    return inputs, np.array(labels, dtype=np.int)

In [ ]:
# 获得训练用的wav文件路径列表 
wav_files = get_wav_files(FLAGS.parent_dir,FLAGS.tr_sub_dirs)
# 获取文件mfcc特征和对应标签
tr_features,tr_labels = extract_features(wav_files)

np.save('tr_features.npy',tr_features)
np.save('tr_labels.npy',tr_labels)

# tr_features=np.load('tr_features.npy')
# tr_labels=np.load('tr_labels.npy')